In [11]:
# Import libraries
import pandas as pd
import sqlite3

In [12]:
# Read teams.csv file
teams = pd.read_csv('teams.csv', index_col=0)

# Connect to SQLite database
conn = sqlite3.connect('fifa_app.db')

# Save teams dataframe to Teams table in SQLite database
teams.to_sql('Teams', conn, if_exists='append', index=True, index_label='Team_ID')

# Close the connection to SQLite database
conn.close()


In [13]:
# Read the stadiums.csv file
stadiums = pd.read_csv("stadiums.csv", index_col=0)

stadiums_json = {
    "name": [
        "Lusail Stadium",
        "Al Bayt Stadium",
        "Khalifa International Stadium",
        "Ahmad Bin Ali Stadium",
        "Education City Stadium",
        "Al Thumama Stadium",
        "Al Janoub Stadium",
        "Stadium 974",
    ],
    "note": [
        "Lusail Iconic Stadium is a sports stadium located in Lusail, Qatar. With a capacity of 80,000 seats, it will serve as the main stadium for the 2022 FIFA World Cup. The stadium is designed to resemble a 'closed shell' and features a unique lighting system that illuminates the stadium at night. It is also equipped with state-of-the-art technology such as video screens and sound systems. The stadium is part of a larger development project in Lusail, which includes residential areas, commercial spaces, and other sports facilities.",
        "Al Bayt Stadium is a sports stadium located in Al Khor City, Qatar. With a capacity of 60,000 seats, it will serve as one of the venues for the 2022 FIFA World Cup. The stadium is designed to resemble a traditional Qatari tent, known as a 'bayt al sha'ar', and features a unique, partially translucent outer layer that allows natural light into the seating area. The stadium is also equipped with state-of-the-art technology such as video screens and sound systems. The stadium is part of a larger development project in Al Khor City, which includes residential areas, commercial spaces, and other sports facilities. The stadium will be transformed after the World Cup into a community sports venue.",
        "Khalifa International Stadium is a sports stadium located in Doha, Qatar. The stadium has undergone a major renovation before the 2022 FIFA World Cup. With a capacity of 40,000 seats, it will serve as one of the venues for the tournament. The stadium is known for its unique design, which features a partial roof that shades the seating area, and also for its state-of-the-art technology such as video screens and sound systems. The stadium was originally built in 1976 and has undergone several renovations and expansions over the years. The latest renovation has made it a modern, air-conditioned facility that meets all FIFA requirements. The stadium will be transformed after the World Cup into a community sports venue and will host various events like concerts, sports competitions, and other entertainment events.",
        "Ahmad Bin Ali Stadium is a sports stadium located in Al Rayyan, Qatar. With a capacity of 40,000 seats, it will serve as one of the venues for the 2022 FIFA World Cup. The stadium is known for its unique design, which features a partially translucent outer layer that allows natural light into the seating area and also for its state-of-the-art technology such as video screens and sound systems. The stadium is part of a larger development project in Al Rayyan, which includes residential areas, commercial spaces, and other sports facilities. The stadium will be transformed after the World Cup into a community sports venue and will host various events like concerts, sports competitions, and other entertainment events.",
        "Education City Stadium is a sports stadium located in Al Rayyan, Qatar. With a capacity of 40,000 seats, it will serve as one of the venues for the 2022 FIFA World Cup. The stadium is known for its unique design, which features a circular shape and a partially translucent outer layer that allows natural light into the seating area and also for its state-of-the-art technology such as video screens and sound systems. Education City Stadium is located in the heart of Education City, Qatar's premier center for higher education, research and innovation. The stadium is part of a larger development project in Education City, which includes residential areas, commercial spaces, and other sports facilities. The stadium will be transformed after the World Cup into a community sports venue and will host various events like concerts, sports competitions, and other entertainment events.",
        "Al Thumama Stadium is a sports stadium located in Al Thumama, Qatar. With a capacity of 40,000 seats, it will serve as one of the venues for the 2022 FIFA World Cup. The stadium is designed to resemble a traditional Qatari headgear, known as a 'gahfiya' and features a unique, partially translucent outer layer that allows natural light into the seating area. The stadium is also equipped with state-of-the-art technology such as video screens and sound systems. The stadium will be transformed after the World Cup into a community sports venue, which will include a sports hall, a gym, and other facilities for the local community.",
        "Al Janoub Stadium is a sports stadium located in Al Wakrah, Qatar. With a capacity of 40,000 seats, it will serve as one of the venues for the 2022 FIFA World Cup. The stadium is known for its unique design, which features a partially translucent outer layer that allows natural light into the seating area and also for its state-of-the-art technology such as video screens and sound systems. The stadium is part of a larger development project in Al Wakrah, which includes residential areas, commercial spaces, and other sports facilities. The stadium was the first of the 8 venues to be completed for the World Cup and was opened in 2019. After the World Cup, the stadium will be transformed into a community sports venue and will host various events like concerts, sports competitions, and other entertainment events.",
        "Stadium 974 is a sports stadium located in Doha, Qatar. The stadium will not be used as a venue for the 2022 FIFA World Cup. The design of the stadium was inspired by the traditional Qatari dhow boat and it has a capacity of 40,000 seats. The stadium will have state-of-the-art technology such as video screens and sound systems. The stadium's design is intended to reflect the country's heritage and culture, while also providing a world-class sports and entertainment venue for the local community. It is planned to be used as a community sports venue after the World Cup, hosting various events like concerts, sports competitions and other entertainment events.",
    ],
}

stadium_dict = {name: note for name, note in zip(stadiums_json["name"], stadiums_json["note"])}

for i, name in enumerate(stadiums["Stadium_name"]):
    stadiums.at[i, "Stadium_description"] = stadium_dict[name]

# Connect to the database, save the dataframe to Stadiums table and close the connection
conn = sqlite3.connect("fifa_app.db")
stadiums.to_sql(
    "Stadiums", conn, if_exists="replace", index=True, index_label="Stadium_ID"
)
conn.close()

In [14]:
import pandas as pd
# Read the players.csv file
players = pd.read_csv('players.csv', index_col=0)

# Replace '-' with ' ' in the Team column
players["Team"].replace("Costa-rica", "Costa Rica", inplace=True)
players["Team"].replace("Saudi-arabia", "Saudi Arabia", inplace=True)

# Normalize the names in the Name column
players['Name'] = players['Name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Create a dictionary with the team names as keys and the team IDs as values
# then map the dictionary to the Team column and drop the Team column
team_dict = dict(zip(teams["Team_name"], teams.index))
players["Team_ID"] = players["Team"].map(team_dict)
players.drop("Team", axis=1, inplace=True)

# Remove the '€' and ',' from the Market_value column
players["Market_value"] = players["Market_value"].str.replace("€", "").str.replace(",", ".")

# Create a function to convert the market value to float
def convert_market_value(value):
    if value.endswith("m"):
        return float(value.rstrip("m")) * 10**6
    elif value.endswith("k"):
        return float(value.rstrip("k")) * 10**3
    else:
        return value

# Apply the function to the Market_value column and convert the column to float
players["Market_value"] = players["Market_value"].apply(convert_market_value)

# Define a function to convert the date format
def convert_date(date_string):
    date_parts = date_string.split(',')
    day_month_year = date_parts[0].split(' ')
    day = day_month_year[1]
    month_name = day_month_year[0]
    year = date_parts[1].split(' ')[1]

    # Convert month name to number
    month_number = {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }[month_name]
    
    if len(day) < 2:
        return f"0{day}-{month_number}-{year}"

    return f"{day}-{month_number}-{year}"

# Apply the function to the 'Date_of_birth' column
players['Date_of_birth'] = players['Date_of_birth'].apply(convert_date)

# Replace the '-' values with the median value and convert the column to int
median_value = players[players["Market_value"] != "-"]["Market_value"].median()
players["Market_value"].replace("-", median_value, inplace=True)
players["Market_value"] = pd.to_numeric(players["Market_value"]).astype(int)

# Replace the values in the Better_foot column with the first letter of the value
# and fill the missing values with 'B'
players["Better_foot"] = players["Better_foot"].map({"right":"R", "left":"L", "both": "B"}).fillna("B")

# Remove the 'cm' and ',' from the Height_cm column and replace the '-' values with the median value
# then convert the column to int
players["Height_cm"] = players["Height_cm"].str.replace(",", "").str.replace("m", "")
median_value = players[players["Height_cm"] != "-"]["Height_cm"].median()
players["Height_cm"].replace("-", median_value, inplace=True)
players['Height_cm'] = players['Height_cm'].astype(int)

# Connect to the database, save the dataframe to Players table and close the connection
conn = sqlite3.connect('fifa_app.db')
players.to_sql('Players', conn, if_exists='append', index=True, index_label='Player_ID')
conn.close()


In [15]:
# Read the matches.csv file
matches = pd.read_csv('matches.csv', index_col=0)

# Replace the '-' with ' ' in the Stadium column, then capitalize the first letter of each word
# and remove ' Iconic' from the end of a stadiums name
matches['stadium'] = matches['stadium'].str.replace("-", " ")
matches['stadium'] = matches['stadium'].str.title()
matches['stadium'] = matches['stadium'].str.replace(" Iconic", "")

# Create a dictionary with the stadium names as keys and the stadium IDs as values
# then map the dictionary to the Stadium column and drop the Stadium column
stadium_dict = dict(zip(stadiums["Stadium_name"], stadiums.index))
matches["Stadium_ID"] = matches["stadium"].map(stadium_dict)
matches.drop("stadium", axis=1, inplace=True)

# Create a dictionary with the team names as keys and the team IDs as values
# then map the dictionary to the Home_team and Away_team columns and drop the columns
home_team_dict = dict(zip(teams["Team_name"], teams.index))
matches["Home_team_ID"] = matches["home_team"].map(home_team_dict)
matches.drop("home_team", axis=1, inplace=True)

away_team_dict = dict(zip(teams["Team_name"], teams.index))
matches["Away_team_ID"] = matches["away_team"].map(away_team_dict)
matches.drop("away_team", axis=1, inplace=True)

# Connect to the database, save the dataframe to Matches table and close the connection
conn = sqlite3.connect('fifa_app.db')
matches.to_sql('Matches', conn, if_exists='append', index=True, index_label='Match_ID')
conn.close()


In [16]:
# Read the events.csv file
events = pd.read_csv('events.csv', index_col=0)

# Create a dictionary with the player names as keys and the player IDs as values
# then map the dictionary to the player column and drop the column
player_dict = dict(zip(players["Name"], players.index))
events["Player_ID"] = events["player"].map(player_dict)
events.drop("player", axis=1, inplace=True)

# Create a dictionary with the home_team-away_team as keys and the match IDs as values
# then map the dictionary to the match column and drop the column
match_dict = {f"{row['home_team']}-{row['away_team']}": index for index, row in pd.read_csv('matches.csv', index_col=0).iterrows()}
events["Match_ID"] = events["match"].map(match_dict)
events.drop("match", axis=1, inplace=True)


# Connect to the database, save the dataframe to Events table and close the connection
conn = sqlite3.connect('fifa_app.db')
events.to_sql('Events', conn, if_exists='append', index=True, index_label='Event_ID')
conn.close()


In [18]:
import sqlite3

conn = sqlite3.connect('fifa_app.db')

cursor = conn.cursor()

cursor.execute('''
    DROP TABLE users
''')

conn.commit()
conn.close()

In [19]:
conn = sqlite3.connect('fifa_app.db')

cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE users (
        User_ID INTEGER PRIMARY KEY,
        Username TEXT NOT NULL,
        Email TEXT NOT NULL,
        Password TEXT NOT NULL
    )
''')

conn.commit()
conn.close()